In [1]:
# !pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]

In [2]:
# !pip install whisper

In [3]:
# !conda install ffmpeg

In [4]:
# !apt-get update && apt-get install -y git-lfs ffmpeg

In [3]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=30,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)

In [6]:
result = pipe("output.mp3")
print(result["text"])

c:\Users\Admin\anaconda3\envs\gumi_env310\lib\site-packages\transformers\models\whisper\generation_whisper.py:483: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
c:\Users\Admin\anaconda3\envs\gumi_env310\lib\site-packages\transformers\models\whisper\modeling_whisper.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Passing a tuple of `past_key_values` is deprecated and will be removed in Tra

 안녕하세요 오늘 날씨가 너무 덥네요.


In [7]:
result = pipe("output.mp3", generate_kwargs = {"language": "english", "task":'translate'})
print(result['text'])

You have passed task=translate, but also have set `forced_decoder_ids` to [[1, None], [2, 50360]] which creates a conflict. `forced_decoder_ids` will be ignored in favor of task=translate.


 Hello. The weather is so hot today.
